## get Cause efffect phrase from the sentence.

In [1]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/ADE_CE_extract_TEST.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

2271

In [2]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

In [3]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [4]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')
###model#gpt-3.5-turbo   gpt-4o

#+ fewshot_examples
def C_E_CP_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=200,
      top_p=1
    )
    
    return response.choices[0].message.content


In [5]:
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]

for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    ######few shot examples augmentation.
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   

    #####################
    #print('fewshot_index :', fewshot_index)
    ##############print('fewshot_examples', fewshot_examples)

    
    
    # output generation
    out = C_E_CP_Identifier(input_sentence, fewshot_examples)
    
    print('out: ', out)
    
    json_out = json.loads(out)
    cause = json_out['cause']
    effect = json_out['effect']
    causality_phrases = json_out['causality_phrase']
    
    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
0
out:  {
  "cause": "long-term therapy with mitomycin C and 5-fluorouracil",
  "effect": "acute renal toxicity",
  "causality_phrase": "in patients receiving"
}
1
out:  {
  "cause": "budesonide",
  "effect": "contact dermatitis",
  "causality_phrase": "causing"
}
2
out:  {
  "cause": "mannitol",
  "effect": "ARF",
  "causality_phrase": "induced"
}
3
out:  {
  "cause": "chlorambucil",
  "effect": "the wbc count began to slowly rise and the patient developed clinical AML",
  "causality_phrase": "was discontinued"
}
4
out:  {
  "cause": "L-asparaginase therapy",
  "effect": "seizures and altered sensorium",
  "causality_phrase": "after"
}
5
out:  {
  "cause": "a regimen of carbamazepine, lynestrenol and sodium valproate",
  "effect": "hepatopathy",
  "causality_phrase": "during"
}
6
out:  {
  "cause": "anastrozole",
  "effect": "sclerosing glomerulonephritis",
  "causality_phrase": "suggesting a newly defined side effect o

out:  {
  "cause": "stopped all medications including the sustained-release verapamil",
  "effect": "her asthma symptoms disappeared",
  "causality_phrase": "and"
}
63
out:  {
  "cause": "administration of clindamycin and trimethoprim-sulfamethoxazole",
  "effect": "cholestatic liver disease with ductopenia (vanishing bile duct syndrome)",
  "causality_phrase": "after"
}
64
out:  {
  "cause": "L-dopa treatment",
  "effect": "rapid mood cycling",
  "causality_phrase": "during"
}
65
out:  {
  "cause": "the first dose of prazosin",
  "effect": "a sudden and severe drop in blood pressure",
  "causality_phrase": "characterized by"
}
66
out:  {
  "cause": "chlorpropamide",
  "effect": "acute intravascular hemolysis",
  "causality_phrase": "developed when"
}
67
out:  {
  "cause": "a higher dose than recommended was used, there was recent or concomitant treatment with mefloquine, there was pre-existing prolongation of the QT interval or the patient had a thiamine deficiency",
  "effect": "8 ca

out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
122
out:  {
  "cause": "Clozapine",
  "effect": "rhabdomyolysis in patients with defective calcium-activated K+ channels",
  "causality_phrase": "is speculated to cause"
}
123
out:  {
  "cause": "rifabutin therapy",
  "effect": "uveitis",
  "causality_phrase": "associated with"
}
124
out:  {
  "cause": "sulfadiazine",
  "effect": "crystalluria and renal failure",
  "causality_phrase": "induced"
}
125
out:  {
  "cause": "isotretinoin (1 mg/kg/day) for cystic acne",
  "effect": "poor night vision and/or excessive glare sensitivity",
  "causality_phrase": "complained of"
}
126
out:  {
  "cause": "calcineurin",
  "effect": "pain syndrome (CIPS)",
  "causality_phrase": "induced"
}
127
out:  {
  "cause": "treatment with olanzapine",
  "effect": "rapidly occurring hyperglycemia",
  "causality_phrase": "that occurred in a geriatric patient 3 days after"
}
128
out:  {
  "cause": "Magnesium tocolysis",
  "effect": "urinary calculu

out:  {
  "cause": "bevacizumab",
  "effect": "spontaneous nasal septal perforation",
  "causality_phrase": "after given"
}
188
out:  {
  "cause": "anastrozole",
  "effect": "renal injury",
  "causality_phrase": "due to"
}
189
out:  {
  "cause": "extemporaneous topical minoxidil",
  "effect": "myocardial infarction",
  "causality_phrase": "died of"
}
190
out:  {
  "cause": "metoclopramide",
  "effect": "Reversible nonthrombocytopenic palpable purpura",
  "causality_phrase": "associated with"
}
191
out:  {
  "cause": "miconazole",
  "effect": "Cardiorespiratory toxicity",
  "causality_phrase": "due to"
}
192
out:  {
  "cause": "BCNU therapy",
  "effect": "fatal pulmonary fibrosis",
  "causality_phrase": "following"
}
193
out:  {
  "cause": "treatment with tamoxifen for breast cancer",
  "effect": "Gastric tumor, endometrial carcinoma and cervical adenocarcinoma in situ were detected",
  "causality_phrase": "after"
}
194
out:  {
  "cause": "disulfiram",
  "effect": "hepatitis",
  "causal

out:  {
  "cause": "fluvoxamine",
  "effect": "akathisia",
  "causality_phrase": "induced"
}
255
out:  {
  "cause": "Allopurinol",
  "effect": "hypersensitivity",
  "causality_phrase": ""
}
256
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
257
out:  {
  "cause": "Copaxone",
  "effect": "Crohn's disease",
  "causality_phrase": "is a potential novel adverse drug effect of"
}
258
out:  {
  "cause": "phenolphthalein",
  "effect": "drug-induced TEN",
  "causality_phrase": "causes of"
}
259
out:  {
  "cause": "Human insulin",
  "effect": "lipoatrophy",
  "causality_phrase": "induced"
}
260
out:  {
  "cause": "propylthiouracil therapy",
  "effect": "antineutrophil cytoplasmic antibody-positive crescentic glomerulonephritis",
  "causality_phrase": "associated with"
}
261
out:  {
  "cause": "levo-dopa",
  "effect": "diphasic dyskinesias",
  "causality_phrase": "induced"
}
262
out:  {
  "cause": "pyridoxine",
  "effect": "seizures",
  "causality_phrase": "dependent"
}
263
out

...

out:  {
  "cause": "15 g of desferrioxamine",
  "effect": "retinal problems",
  "causality_phrase": "after only"
}
1966
out:  {
  "cause": "carbamazepine administration",
  "effect": "syndrome of inappropriate antidiuresis",
  "causality_phrase": "as a much earlier side-effect of"
}
1967
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
1968
out:  {
  "cause": "procainamide therapy",
  "effect": "clinical lupus syndrome with positive antinuclear antibody, positive lupus erythematosus (LE) cell preparation, and diffuse proliferative glomerulonephritis",
  "causality_phrase": "following 26 months of"
}
1969
out:  {
  "cause": "lithium therapy",
  "effect": "thyrotoxicosis with severe exophthalmos",
  "causality_phrase": "while on"
}
1970
out:  {
  "cause": "treatment with carbamazepine and diphenylhydantoin",
  "effect": "deposits of plasma proteins in the skin",
  "causality_phrase": "during"
}
1971
out:  {
  "cause": "imatinib",
  "effect": "tumor lysis syndrome",
  "ca

out:  {
  "cause": "the combination of rifabutin, clarithromycin and ethambutol",
  "effect": "uveitis",
  "causality_phrase": "during treatment of disseminated Mycobacterium avium-intracellulare complex infection with"
}
2027
out:  {
  "cause": "high doses of trazodone",
  "effect": "movement disorder",
  "causality_phrase": "following"
}
2028
out:  {
  "cause": "Cefoxitin therapy",
  "effect": "granulomatous hepatitis",
  "causality_phrase": "associated"
}
2029
out:  {
  "cause": "the tyrosine kinase inhibitor sunitinib",
  "effect": "Biopsy-proven acute interstitial nephritis",
  "causality_phrase": "associated with"
}
2030
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2031
out:  {
  "cause": "nifedipine treatment",
  "effect": "rupture of a cerebral aneurysm",
  "causality_phrase": "associated with"
}
2032
out:  {
  "cause": "Methotrexate",
  "effect": "diffuse interstitial pulmonary fibrosis",
  "causality_phrase": "induced"
}
2033
out:  {
  "cause": "fludarabi

out:  {
  "cause": "nitrofurantoin",
  "effect": "minor or life-threatening pulmonary dysfunction",
  "causality_phrase": "can cause"
}
2091
out:  {
  "cause": "therapy for HLH",
  "effect": "etoposide-related secondary acute myeloid leukemia (sAML)",
  "causality_phrase": "following"
}
2092
out:  {
  "cause": "acetazolamide",
  "effect": "Severe acidosis",
  "causality_phrase": "from"
}
2093
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2094
out:  {
  "cause": "two instillations of epirubicin",
  "effect": "significant morbidity",
  "causality_phrase": "related to"
}
2095
out:  {
  "cause": "various medications, such as Haloperidol",
  "effect": "NMS",
  "causality_phrase": "is a drug-related response to"
}
2096
out:  {
  "cause": "the antitumor necrosis factor-alpha blocking agent etanercept",
  "effect": "persistent hypoglycemia",
  "causality_phrase": "experienced"
}
2097
out:  {
  "cause": "ibuprofen",
  "effect": "meningitis",
  "causality_phrase": "induced"
}

out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2150
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2151
out:  {
  "cause": "lithium",
  "effect": "CDI",
  "causality_phrase": "associated"
}
2152
out:  {
  "cause": "praziquantel administration",
  "effect": "production of marked inflammation",
  "causality_phrase": "as a result of"
}
2153
out:  {
  "cause": "ergotamine",
  "effect": "arteriospasm",
  "causality_phrase": "induced"
}
2154
out:  {
  "cause": "initial infusion of rituximab",
  "effect": "acute coronary syndromes",
  "causality_phrase": "associated with"
}
2155
out:  {
  "cause": "a high dose combined with a short infusion time",
  "effect": "anaphylactoid reactions",
  "causality_phrase": "increases the risk of"
}
2156
out:  {
  "cause": "ferrous sulfate",
  "effect": "hypothyroid",
  "causality_phrase": "while taking"
}
2157
out:  {
  "cause": "valproic acid",
  "effect": "dementia",
  "causality_phrase": "induced"
}
2158
out:  {
  "ca

out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2215
out:  {
  "cause": "dexamethasone",
  "effect": "hypertrophic obstructive cardiomyopathy in premature infants",
  "causality_phrase": "induced"
}
2216
out:  {
  "cause": "foscarnet",
  "effect": "electrolyte disorders",
  "causality_phrase": "associated with"
}
2217
out:  {
  "cause": "quinapril therapy",
  "effect": "a dry, irritating cough",
  "causality_phrase": "within one month of starting"
}
2218
out:  {
  "cause": "ifosfamide",
  "effect": "lethal anuria",
  "causality_phrase": "was mainly due to"
}
2219
out:  {
  "cause": "isoniazid",
  "effect": "pellagra",
  "causality_phrase": "may induce"
}
2220
out:  {
  "cause": "cisplatin",
  "effect": "C. difficile colitis",
  "causality_phrase": "may cause"
}
2221
out:  {
  "cause": "",
  "effect": "",
  "causality_phrase": ""
}
2222
out:  {
  "cause": "cimetidine",
  "effect": "hepatitis",
  "causality_phrase": "induced"
}
2223
out:  {
  "cause": "omeprazole therap

In [6]:
len(filtered_rows)

2271

In [7]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [8]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",,,
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,in patients receiving
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol,ARF,induced
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",chlorambucil,the wbc count began to slowly rise and the pat...,was discontinued
...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",,,
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...","a second course of flurbiprofen, a nonsteroida...",biopsy-proven acute tubulointerstitial nephritis,associated with
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,male gynaecomastia,treated with


In [9]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [10]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        cause_pred = filtered_rows['cause'][i].lower()
        cause_t = filtered_rows['cause_t'][i].lower()
        
        effect_pred = filtered_rows['effect'][i].lower()
        effect_t = filtered_rows['effect_t'][i].lower()

        # Slide through input_sentence with a window size of 8
        #print('effect_pred',  effect_pred)
        #print('effect_t',  effect_t)


        score = get_score(cause_t, cause_pred)################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t, effect_pred)################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [11]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

no pred_phrase
no pred_phrase
prediction length lower less gastrointestinal intolerance p: acute and chronic pulmonary reactions
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
prediction length lower chronic renal failure p: acute hemorrhagic gastritis
prediction length lower hypercalcemia p: vitamin d intoxication
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
prediction length lower clinical improvement p: enterococcal peritonitis
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phrase
no pred_phra

In [12]:
sum(prediction)/len(filtered_rows)

0.7855570233377367

In [13]:
sum(c_prediction)/len(filtered_rows)

0.8383971818582122

In [14]:
sum(e_prediction)/len(filtered_rows)

0.8199031263760458

In [15]:
filtered_rows['predictions'] = prediction

In [16]:
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase,predictions
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",,,,0
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,in patients receiving,1
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing,1
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol,ARF,induced,1
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",chlorambucil,the wbc count began to slowly rise and the pat...,was discontinued,1
...,...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",,,,0
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to,0
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...","a second course of flurbiprofen, a nonsteroida...",biopsy-proven acute tubulointerstitial nephritis,associated with,1
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,male gynaecomastia,treated with,1


In [17]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT3/KNN_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [18]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
